In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial.distance import cosine
import scipy
from ggplot import *


import object_class as oc
from object_vectors import *
import compare_vectors as cv

In [10]:
wordvecs = oc.wordvecs(seedlings_categories, "../../../../object_centrality/data_local/dict_glove_42b_300", 
                                             "../../../../object_centrality/data_local/glove_unit_42b_300.npy")

In [11]:
wordvecs.sort(key=lambda x: x[0])
columns = [x[0] for x in wordvecs]

pairwise_matrix = []
for vec in wordvecs:
    result = []
    for other in wordvecs:
        result.append(cosine(vec[1], other[1]))
    pairwise_matrix.append(result)
# pairwise_word = pd.DataFrame(pairwise_matrix, columns=columns, index=columns)
# pairwise_word.to_csv("pairwise_word_distances.csv")

In [13]:
image_vecs = []

for x in seedlings_categories:
    result = oc.find_central_vector(x)
    image_vecs.append((x, result[0]))
    
image_vecs.sort(key=lambda x: x[0])
pairwise_image_matrix = []
for vec in image_vecs:
    result = []
    for other in image_vecs:
        result.append(cosine(vec[1], other[1]))
    pairwise_image_matrix.append(result)
    
# columns = ['word', 'other_word', 'cosine']

# pairwise_image = pd.DataFrame(pairwise_image_matrix, columns=columns, index=columns)
# pairwise_image.to_csv("pairwise_image_distances.csv")

In [42]:
animate = ["puppy", "duck", "elephant", "pig", "monkey", "giraffe",
             "bear", "baby", "fish", "frog", "cow", "dog", "cat"]
results = []
for idx1, word1 in enumerate(sorted(seedlings_categories)):
    img_vec1 = image_vecs[idx1]
    wrd_vec1 = wordvecs[idx1]
    for idx2, word2 in enumerate(sorted(seedlings_categories)):
        img_vec2 = image_vecs[idx2]
        wrd_vec2 = wordvecs[idx2]
        anim = "mixed"
        if word1 in animate and word2 in animate:
            anim = "animate"
        elif word1 not in animate and word2 not in animate:
            anim = "inanimate"
        if word1 != word2:
            results.append([word1, word2, cosine(img_vec1[1], img_vec2[1]), cosine(wrd_vec1[1], wrd_vec2[1]), anim])
        
pairwise = pd.DataFrame(results, columns=["X1", "word", "cos_img", "cos_word", "animate"])
pairwise.to_csv("pairwise_distances.csv", index=False)

In [43]:
df2 = pd.DataFrame()

for idx, group in pairwise.groupby("word"):
    group["zscore_img"] = scipy.stats.zscore(group.cos_img)
    group["zscore_word"] = scipy.stats.zscore(group.cos_word)
    df2 = df2.append(group)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [44]:
overlaps = df2.query("(zscore_word <= -1) & (zscore_img <= -1)")
overlaps.to_csv("overlapping_pairwise.csv", index=False)

In [45]:
words = {}

for idx, item in df2.iterrows():
    avg_zscore = round(float((item.zscore_img+item.zscore_word))/2, 2)
    if item.word not in words:
        if item.zscore_img <= -1:
            words[item.word] = [[(item.X1, avg_zscore)],[]]
        if item.zscore_word <= -1:
            words[item.word] = [[],[(item.X1, avg_zscore)]]
    else:
        if item.zscore_img <= -1:
            words[item.word][0].append((item.X1, avg_zscore))
        if item.zscore_word <= -1:
            words[item.word][1].append((item.X1, avg_zscore))

In [46]:
overlap_ratios = {}
for key, values in words.items():
    intersect = set(values[0]) & set(values[1])
    n = len(intersect)
    overlap_ratios[key] = [round(float(n)/len(set(values[0]+values[1])), 3), 
                           [list(x) + [True] for x in intersect] +
                           [list(x) + [False] for x in values[0]+values[1] if x not in intersect]]
    

In [47]:
overlap_ratios_list = [(x, y[0], sorted(list(y[1]), key=lambda x: x[1])) for x, y in overlap_ratios.items()]

In [48]:
overlap_table = pd.DataFrame(overlap_ratios_list, columns = ["word", "overlap_ratio", "neighbors"])

In [114]:
overlap_table.sort_values(by="word").to_csv("overlap_table.csv", index=False)